<a href="https://colab.research.google.com/github/Kennedy87670/NLP/blob/main/Introduction_to_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# introduction to NLP Fundamentals in Tensorflow

NLP has the goal of deriving information out of natural language (could be sequences text or speech).

Another common term for NLP problems is sequence to sequence problems (seq2seq).

# Check our GPU

In [2]:
!nvidia-smi - l

Invalid combination of input arguments. Please run 'nvidia-smi -h' for help.



In [3]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

#import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2022-08-25 12:23:52--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-08-25 12:23:52 (97.4 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get a text dataset

The dataset we are going to be using is Kaggle's introduction to NLP dataset (text samples of tweets labeled as diaster or not diaster).

See the original source here https://www.kaggle.com/competitions/nlp-getting-started

In [4]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

#unzip data
unzip_data('nlp_getting_started.zip')

--2022-08-25 12:23:55--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.24.128, 142.251.10.128, 142.251.12.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.24.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.007s  

2022-08-25 12:23:56 (82.3 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



# Visualize a text dataset


In [5]:
import pandas as pd

train_df = pd.read_csv('train.csv')
test_df =pd.read_csv('test.csv')

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
# shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
# what does the test dataframe look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
#how many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [9]:
# how many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [10]:
# lets visualize some random samples
import random
random_index = random.randint(0, len(train_df)-5) #create random indexes
for row in train_df_shuffled[['text', 'target']][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target:{target}", '(real diaster)' if target >0 else '(not real diaster' )
  print(f'Text:\n{text}\n')
  print('---\n')

Target:0 (not real diaster
Text:
@czallstarwes more like demolition derby ??

---

Target:0 (not real diaster
Text:
Souda leave Lady Sonia alone or I shall obliterate you #KneelBot

---

Target:0 (not real diaster
Text:
'Sometimes God uses sorrowful tragedy to set the stage for glorious redemption.' -David Platt Run forÛ_ https://t.co/86V81dv00E

---

Target:1 (real diaster)
Text:
IJ: Texas Seeks Comment on Rules for Changes to Windstorm Insurer http://t.co/h132iuL7MU

---

Target:0 (not real diaster
Text:
School Of Seven Bells - Windstorm  http://t.co/E1kbluDwh5 #nowplaying

---



In [11]:
## split data into training and validation dataset
from sklearn.model_selection import train_test_split

In [12]:
# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # dedicate 10% of samples to validation set
                                                                            random_state=42)

In [13]:
# Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)


(6851, 6851, 762, 762)

In [14]:
# check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

## Converting text into numbers
 
when dealing with text problem, one of the first things you will have to do before you can build a model is to convert your text to numbers.

There are a few ways to do this, namely:
* Tokenziation - direct mapping of token (a token could be a word or a character) to number
* Embedding - create a matrix of feature vecto for each token (the size of the feature vetor can be defined and this embedding can be learned)

## Text Vectorization (tokenization)

In [15]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [16]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization variables
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (all of the different words in your text)
                                    standardize="lower_and_strip_punctuation", # how to process text
                                    split="whitespace", # how to split tokens
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None) # how long should the output sequence of tokens be?
                                    # pad_to_max_tokens=True) # Not valid if using max_tokens=None


In [17]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [18]:
# setup text vectorization variable
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length= 15  # max length our sequences will be (e.g how many words from a tweet does a model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode='int',
                                    output_sequence_length=max_length)

In [19]:
# fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)


In [20]:
# create a sample sentence and tokenize it
sample_sentence = "Theres a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [21]:
# choose a random sentence from the training datset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"original text: \n {random_sentence}\
\n\nVectorized version: ")
text_vectorizer([random_sentence])

original text: 
 -honey you ain't no angel. You like to scream these words as a weapon. Well go ahead take your best shot woman. I wanna leave you it's

Vectorized version: 


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[3775,   12,  865,   40, 1905,   12,   25,    5, 4675,  222,  590,
          26,    3,  263,  251]])>

In [23]:
# get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()  #get all the unique words in our training data 
top_5_words = words_in_vocab[:5] # get the most common words
bottom_5_words = words_in_vocab[-5:] # get the least common words
print(f"NMumber of words in vocab: {len(words_in_vocab)}")
print(f" 5 most common words:{top_5_words} ")
print(f" 5 most common words:{bottom_5_words} ")

NMumber of words in vocab: 10000
 5 most common words:['', '[UNK]', 'the', 'a', 'in'] 
 5 most common words:['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1'] 


## Creatign an Embedding using Embedding layer

To make our embedding, we are going to use Tensorflow's embedding layers: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding

The parameters we care most about for our embedding layer:
* Input dim = the size of our get_vocabulary
* output dim = the size of the output embedding vector, for example, a value of 100 would mean token get represented by a vector 100 long
* input_length = length of sequences passed to the embedding layers

In [28]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             embeddings_initializer='uniform',
                             output_dim=128,
                             input_length= max_length #how long each input
                             )
embedding

In [29]:
# Get a random sentence from the training set
random_sentence =random.choice(train_sentences)
print(f"Original text:\n {random_sentence} ")

# embedd the random sentence (turn it into dense vectors of fixed size)
sample_embed= embedding(text_vectorizer([random_sentence]))

sample_embed

Original text:
 Virgin galactic crash: early unlocking of brakes triggered structural failure - Irish Examiner http://t.co/ocMCvfDZkv 


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.02885997,  0.00445544, -0.00241885, ...,  0.03657711,
          0.0451124 ,  0.03016507],
        [-0.02382345,  0.04868654, -0.03380114, ...,  0.04854554,
          0.02689389,  0.03206427],
        [-0.02769746, -0.03288432, -0.03689475, ..., -0.02039436,
          0.02131626,  0.01162617],
        ...,
        [ 0.0015739 , -0.03747381, -0.04334254, ..., -0.04239656,
         -0.04745188, -0.02723811],
        [ 0.01112573,  0.00946088,  0.03607413, ...,  0.00131917,
          0.0226818 , -0.00323661],
        [ 0.01112573,  0.00946088,  0.03607413, ...,  0.00131917,
          0.0226818 , -0.00323661]]], dtype=float32)>

In [32]:
# Check out a single token embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence[0]

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.02885997,  0.00445544, -0.00241885, -0.00313202,  0.04233198,
         0.01684654,  0.01609954,  0.01940167,  0.00624361,  0.04736925,
        -0.03182502,  0.03522538, -0.00344404,  0.00582061,  0.03251228,
         0.02118406, -0.00306188, -0.03953296, -0.02458714, -0.02748888,
         0.04878099,  0.00294274,  0.02371386,  0.03378363, -0.01359384,
        -0.03217906, -0.03013078, -0.04323324, -0.04560733, -0.03254181,
        -0.04318638, -0.0427143 ,  0.02813864,  0.00315291, -0.04358457,
        -0.00236094, -0.02824422,  0.01965697,  0.035832  ,  0.02102819,
        -0.00721546,  0.04513413, -0.04226102,  0.01903588, -0.01438919,
        -0.02833664,  0.00321168,  0.02576797,  0.02497046,  0.01894608,
        -0.01592332,  0.00334001,  0.04966312, -0.02600026,  0.04710765,
         0.02722621,  0.01960523,  0.00086337, -0.03266571,  0.02274389,
         0.01406339,  0.03571391,  0.04451028, -0.04699378,  0.03614572,
  